# Sequential Tool Execution with Vector Search

Understanding how LangGraph executes dependent tools where search results inform subsequent calculations.

## Learning Objectives

By the end of this notebook, you will:

1. **Understand sequential dependencies** - Recognize when tasks have dependencies where one tool's output becomes another tool's input
2. **Identify sequential patterns** - Recognize multiple separate AIMessages with tool_calls indicating sequential loops through the graph
3. **Trace data flow** - Verify how the LLM extracts values from vector search results and uses them as parameters in calculation tools
4. **Apply to RAG workflows** - Combine semantic search with computational tools to create practical applications

## 1. Environment Setup

In [ ]:
# Core imports
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.documents import Document
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

from dotenv import load_dotenv
from typing import Literal, List

load_dotenv("../../.env")
print("✅ Environment loaded")

In [ ]:
# Initialize LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens=1024
)

print("✅ LLM initialized")

## 2. Connect to Vector Store

In [ ]:
# ChromaDB Configuration
PERSIST_DIR = "../../chroma_db"
COLLECTION_NAME = "toyota_specs"
EMBED_MODEL_ID = "gemini-embedding-001"

# Initialize embeddings
embeddings_model = GoogleGenerativeAIEmbeddings(
    model=EMBED_MODEL_ID,
    output_dimensionality=768
)

# Connect to vectorstore
vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    embedding_function=embeddings_model,
    persist_directory=PERSIST_DIR
)

print(f"✅ Connected to vectorstore: {COLLECTION_NAME}")
print(f"   Total documents: {vectorstore._collection.count()}")

## 3. Define Tools

In [ ]:
# Vector similarity search helper
def vector_similarity_search(
    query: str, 
    vectorstore, 
    k: int = 5
) -> List[str]:
    """Perform vector similarity search."""
    docs = vectorstore.similarity_search(query, k=k)
    return [doc.page_content for doc in docs]

In [ ]:
# Tool 1: Vehicle Search
@tool
def search_vehicles(query: str, max_results: int = 5) -> str:
    """
    Search Toyota vehicle database using semantic similarity.
    
    Use this tool when users need information about Toyota vehicles,
    including specifications, pricing, fuel efficiency, or comparisons.
    
    Args:
        query: Natural language search query about Toyota vehicles
        max_results: Maximum number of results to return (default: 5)
    
    Returns:
        Formatted string with vehicle information
    """
    docs = vector_similarity_search(query, vectorstore, k=max_results)
    
    result = "Vehicle Search Results:\n"
    result += "=" * 60 + "\n"
    for i, doc in enumerate(docs, 1):
        result += f"\nResult {i}:\n{doc}\n"
    result += "=" * 60
    
    return result

print("✅ search_vehicles tool defined")

In [ ]:
# Tool 2: EMI Calculator
@tool
def emi_calculator(principal: float, annual_interest_rate: float, tenure_months: int, currency: str) -> str:
    """
    Calculate the EMI (Equated Monthly Installment) for a loan.
    
    Use this tool when users want to know their monthly loan payment,
    total repayment amount, or total interest for a loan.
    
    Args:
        principal: The loan amount
        annual_interest_rate: Annual interest rate as percentage (e.g., 8.5)
        tenure_months: Loan tenure in months
        currency: Currency code (USD, EUR, GBP, INR, JPY)
    """
    if principal <= 0 or annual_interest_rate < 0 or tenure_months <= 0:
        return "Error: Invalid input parameters"
    
    monthly_interest_rate = annual_interest_rate / 12 / 100
    
    if monthly_interest_rate == 0:
        emi = principal / tenure_months
        total_payment = principal
        total_interest = 0
    else:
        emi = principal * monthly_interest_rate * \
              pow(1 + monthly_interest_rate, tenure_months) / \
              (pow(1 + monthly_interest_rate, tenure_months) - 1)
        total_payment = emi * tenure_months
        total_interest = total_payment - principal
    
    return (
        f"EMI Calculation Result:\n"
        f"  Loan Amount: {principal:,.2f} {currency}\n"
        f"  Interest Rate: {annual_interest_rate}% per annum\n"
        f"  Tenure: {tenure_months} months\n"
        f"  Monthly EMI: {emi:,.2f} {currency}\n"
        f"  Total Payment: {total_payment:,.2f} {currency}\n"
        f"  Total Interest: {total_interest:,.2f} {currency}"
    )

print("✅ emi_calculator tool defined")

## 4. Build LangGraph Workflow

In [ ]:
# Initialize LLM with tools
tools = [search_vehicles, emi_calculator]
llm_with_tools = llm.bind_tools(tools)

def call_llm(state: MessagesState):
    """LLM node: Calls LLM with current messages."""
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """Router: Check if agent wants to use tools."""
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    return END

# Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("llm", call_llm)
workflow.add_node("tools", ToolNode(tools))
workflow.add_edge(START, "llm")
workflow.add_conditional_edges("llm", should_continue, {"tools": "tools", END: END})
workflow.add_edge("tools", "llm")

app = workflow.compile()
print("✅ Graph compiled")

## 5. Understanding Sequential Execution

### What Makes Execution Sequential?

Sequential execution occurs when tasks have **dependencies** - one task's output is needed as input for the next task.

**Key Indicators in Queries:**
- **"then"** - Explicitly signals sequential order ("find price, then calculate EMI")
- **"that amount/price"** - References result from previous task
- **"for it/that"** - Pronoun referring to previous result

**Our Example Query:**

*"Find the base price of the Toyota Camry, **then** calculate the EMI for **that amount** in USD at 7% interest for 60 months"*

This query has a clear dependency:
1. First: Search for Camry price
2. Then: Use that price to calculate EMI

The LLM cannot calculate EMI until it knows the price from the search!

## 6. Execute Sequential Workflow

In [ ]:
# Sequential test: Dependent tasks
sequential_state = {
    "messages": [
        HumanMessage(content="Find the base price of the Toyota Camry, then calculate the EMI for that amount in USD at 7% interest for 60 months")
    ]
}

print("SEQUENTIAL EXECUTION TEST")
print("=" * 80)
print("Query: Find Camry base price, THEN calculate its EMI")
print("\nExpected Pattern: MULTIPLE SEPARATE AIMessages with tool_calls")
print("=" * 80)

In [ ]:
# Execute sequential test
sequential_result = app.invoke(sequential_state)
print(f"\nTotal messages: {len(sequential_result['messages'])}")

## 7. Verify Sequential Pattern

In sequential execution, we expect:
- **Multiple AIMessages** with tool_calls (not just one)
- Each AIMessage represents a separate loop through the graph
- Pattern: AIMessage₁ → ToolMessage₁ → AIMessage₂ → ToolMessage₂ → AIMessage₃ (final)

In [ ]:
# Verify sequential execution
tool_call_messages = [
    msg for msg in sequential_result['messages']
    if isinstance(msg, AIMessage) and hasattr(msg, 'tool_calls') and msg.tool_calls
]

print("SEQUENTIAL EXECUTION VERIFICATION")
print("=" * 80)
print(f"AIMessages with tool_calls: {len(tool_call_messages)}")

if len(tool_call_messages) > 1:
    print("\n🔄 CONFIRMED: Sequential execution detected!")
    print(f"   {len(tool_call_messages)} SEPARATE tool call requests\n")
    
    for i, msg in enumerate(tool_call_messages, 1):
        print(f"  Loop {i}: {msg.tool_calls[0]['name']}")
        print(f"    Args: {msg.tool_calls[0]['args']}\n")
else:
    print("\n⚠️ Single tool call message (might be parallel)")

## 8. Trace Data Flow

The key to sequential execution is how data flows between tools:
1. **Search executes** → Returns vehicle pricing information
2. **LLM extracts** → Parses "$26,000" from the search result
3. **EMI called** → Uses extracted price as principal parameter

In [ ]:
# Verify data flow
print("DATA FLOW VERIFICATION")
print("=" * 80)

if len(tool_call_messages) >= 2:
    # Get the vehicle search result
    search_result = sequential_result['messages'][2]  # First ToolMessage
    print("STEP 1 - Vehicle Search Result:")
    print(f"  {search_result.content[:200]}...\n")
    
    # Get the EMI tool call
    emi_call = sequential_result['messages'][3]  # Second AIMessage
    print("STEP 2 - EMI Calculator Called With:")
    for key, value in emi_call.tool_calls[0]['args'].items():
        print(f"  {key}: {value}")
    
    print("\n" + "-" * 80)
    print("✅ LLM extracted price from search and used it for EMI calculation!")

## 9. Examine Message Sequence

Let's examine each message in the sequence to understand the flow:

In [ ]:
# Show all messages
sequential_result['messages']

In [ ]:
# Message 0: User query
dict(sequential_result['messages'][0])

In [ ]:
# Message 1: First AIMessage - calls search_vehicles
dict(sequential_result['messages'][1])

In [ ]:
# Message 2: First ToolMessage - search result
dict(sequential_result['messages'][2])

In [ ]:
# Message 3: Second AIMessage - calls emi_calculator with extracted price
dict(sequential_result['messages'][3])

In [ ]:
# Message 4: Second ToolMessage - EMI result
dict(sequential_result['messages'][4])

In [ ]:
# Message 5: Third AIMessage - final response synthesizing both results
dict(sequential_result['messages'][5])

## 10. Final Response

In [ ]:
# Show final response
print("FINAL RESPONSE:")
print("=" * 80)
print(sequential_result['messages'][-1].content)
print("=" * 80)

## 11. Streaming Execution

Watch the sequential loops happen in real-time through streaming.

In [ ]:
# Streaming view
state_stream = {
    "messages": [
        HumanMessage(content="Find the base price of the Toyota Camry, then calculate the EMI for that amount in USD at 7% interest for 60 months")
    ]
}

print("STREAMING EXECUTION")
print("=" * 80)
print("Watch the MULTIPLE loops in sequential execution...\n")

step_count = 0
loop_count = 0

for event in app.stream(state_stream):
    for node_name, data in event.items():
        step_count += 1
        print(f"\n[Step {step_count}] Node: '{node_name}'")
        print("-" * 60)
        
        if "messages" in data:
            for msg in data["messages"]:
                if isinstance(msg, AIMessage):
                    if hasattr(msg, "tool_calls") and msg.tool_calls:
                        loop_count += 1
                        print(f"  🔄 LOOP {loop_count}: Calling {msg.tool_calls[0]['name']}")
                        print(f"     Args: {msg.tool_calls[0]['args']}")
                    else:
                        print(f"  💬 Final response generated")
                        
                elif isinstance(msg, ToolMessage):
                    print(f"  ✅ Tool executed")
                    first_line = msg.content.split('\n')[0]
                    print(f"     Result: {first_line}")

print("\n" + "=" * 80)
print(f"Total steps: {step_count}")
print(f"Total loops (agent → tools): {loop_count}")
print("=" * 80)

## 12. Additional Example: Another Sequential Query

Test another sequential pattern to reinforce understanding.

In [ ]:
# Another sequential example
state2 = {
    "messages": [
        HumanMessage(content="What's the starting price for the RAV4 Hybrid? Then calculate monthly payments for that price at 5.9% over 72 months.")
    ]
}

print("SECOND SEQUENTIAL TEST")
print("=" * 80)
print("Query: RAV4 Hybrid price → EMI calculation")
print("=" * 80)

result2 = app.invoke(state2)

print(f"\nTotal messages: {len(result2['messages'])}")
print("\nFinal Response:")
print("-" * 80)
print(result2['messages'][-1].content)
print("=" * 80)

## Conclusion

In this notebook, you learned:

✅ **Sequential execution dependencies** - When tasks have dependencies ("then", "that amount"), the LLM executes tools in separate loops, using results from one tool to inform parameters for the next

✅ **Sequential pattern recognition** - Multiple separate AIMessages with tool_calls (e.g., AIMessage₁ → ToolMessage₁ → AIMessage₂ → ToolMessage₂ → AIMessage₃) indicating 2+ loops through the graph, resulting in 6+ total messages

✅ **Data flow tracing** - The LLM extracts specific values from ToolMessage content (e.g., "$26,000" from search results) and uses them as parameters in subsequent tool calls (e.g., `principal=26000` for EMI calculation)

✅ **RAG + Computation pattern** - Combined vector similarity search (information retrieval) with financial calculations (processing) to create a practical car-buying assistant that handles dependent tasks intelligently

### Key Insights

**Message Count**: Sequential execution typically produces 6+ messages (vs 5 for parallel) due to multiple loops:
- HumanMessage (query)
- AIMessage₁ (call tool 1)
- ToolMessage₁ (tool 1 result)
- AIMessage₂ (call tool 2 with extracted data)
- ToolMessage₂ (tool 2 result)
- AIMessage₃ (final synthesis)

**LLM Intelligence**: The LLM demonstrates semantic understanding by:
- Recognizing dependencies in natural language ("then", "that amount")
- Parsing and extracting relevant values from unstructured text
- Mapping extracted values to correct tool parameters

**Real-World Applications**:
- E-commerce: Search for product → Calculate shipping cost
- Real estate: Find property → Calculate mortgage
- Travel: Search flights → Estimate total trip cost
- Healthcare: Look up medication → Calculate dosage

### Next Steps

Now that you understand sequential execution, you're ready to explore **conversational context management** where the agent maintains state across multiple user turns for natural multi-turn interactions.